# Extracting multimodal data features

`siibra` provides access to data features of different modalities using the `get_features` method, which accepts a feature modality and is sensitive to the selections configured in the atlas (parcellation, region). If not particular selection is made, `get_features` considers all brain regions of the current parcellation.

In [ ]:
import siibra
from os import environ

### Prerequisite: Allow  `siibra` to connect to the EBRAINS Knowledge Graph

Most of the features linked to atlas regions are curated datasets in the [EBRAINS knowledge graph](https://kg.ebrainseu) (KG). To fetch data from the KG, you need to pass an EBRAINS KG authentication token to siibra. For being able to generate such tokens, you need to follow three steps:

1. Register for an EBRAINS account, as described [here](https://ebrains.eu/register).
2. Enable API access for your EBRAINS account, as described [here](https://kg.humanbrainproject.eu/develop.html)).
3. Generate a token by visiting the [KG authorization endpoint](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize).

Step 1. and 2. need to be performed only once. Step 3 has to be performed every once in a while, since the KG does not (yet) provide persistent tokens. 

Just copy the token text from the resulting page, and run the next cell to paste it in.

In [ ]:
token = input("Enter your token here, then press 'Enter': ")
siibra.set_ebrains_token(token)

### Densities of neurotransmitter receptors

Transmitter receptor density fingerprints are linked to brain regions by their name in the EBRAINS Knowledge Graph. Like any data feature, they are accessed using the `get_features` method of the atlas, which makes use of the current selection in the atlas. The `get_features` method knows from the specified data modality that the match is determined from the brain region identified. Receptor densities come as a nicely structured datatype. Amongst other things, they can visualize themselves in a plot.

If we don't specify a particular selection, the atlas will return all available receptor density features linked to the parcellation map:

In [ ]:
atlas = siibra.atlases['human']
atlas.select_parcellation('julich')
atlas.clear_selection()
features = atlas.get_features(
    siibra.modalities.ReceptorDistribution)
print("Receptor density features found for the following regions:")
print(", ".join({f.regionspec for f in features}))

If we select a region, the returned list is filtered accordingly. 

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(
    siibra.modalities.ReceptorDistribution)
for r in features:
    fig = r.plot(r.regionspec)

### Layer-specific cell distributions in cortical regions

This version of siibra provides a pre-release preview of layer-specific cell distributions in different areas of BigBrain.

In [ ]:
atlas = siibra.atlases['human']
atlas.select_parcellation('julich')
atlas.select_region("V1")
features = atlas.get_features(
    siibra.modalities.CorticalCellDistribution)

for index in [0,-1]:
    # we plot only the first and last feature here
    fig = features[index].plot(features[index].regionspec)


### Gene Expressions from the Allen Atlas 

The atlas client can make calls to gene expression data from the Allen atlas and evaluate them in the ICBM space to find regional gene expression levels. It also has a list of available gene names for convenient selection. Gene expressions are linked to atlas regions by coordinates of their probes in MNI space. The `get_features` method detects this from the feature modality, and applies the mask of the regions that are currently selected in the atlas to filter the probes. We can visualize these filtered locations.

In [ ]:
atlas.select_region("V1")
features = atlas.get_features(
    siibra.modalities.GeneExpression, 
    gene=siibra.features.gene_names.GABARAPL2)
print(features[0])

# plot
from nilearn import plotting
all_coords = [tuple(g.location) for g in features]
mask = atlas.build_mask(siibra.spaces.MNI152_2009C_NONL_ASYM)
display = plotting.plot_roi(mask)
display.add_markers(all_coords,marker_size=5) 

### Connectivity matrices
For `siibra`, a connectivity matrix is a data feature like the others below, and can be found using the same `get_features` function just by choosing another modality. `siibra` knows from the modality type that this type of data does not match to the selected brain region in the atlas, but to the selected parcellation. Therefore, `get_features` return connectivity datasets that are defined for the selected parcellation. 

To learn about the nature of the provided connectivity, the `src_info` attribute provides a detailed description of the dataset. 

In [ ]:
# Get the first four connectivity matrices available for the parcellation
features = atlas.get_features(siibra.modalities.ConnectivityMatrix)[:4]

# format dataset names for use as figure titles
from textwrap import wrap
titleformat = lambda text : "\n".join(wrap(text.replace('_',' '),20)) 

# plot the matrices
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure()
for i,feature in enumerate(features):
    ax = fig.add_subplot(1,len(features),i+1)
    ax.imshow(np.log(feature.array),cmap=plt.cm.hot)
    ax.set_title(titleformat(feature.src_name),size=10)
fig.show()

### Locations of iEEG recordings

iEEG data is queried per session/subject. The data is organized into multiple electrodes per session, and multiple contact points per electrode. Iterating over a session yields its electrodes. Iterating over each electrode yields its contact points. Each of these items are themselves of type `SpatialFeature`, so they have a `.location` and can be matched against an atlas. Below, we use this to visualize MNI coordinates of contact points which correspond to electrodes touching the selected region. We highlight the contact points inside the selected atlas region in red.

In [ ]:
sessions = atlas.get_features(siibra.modalities.IEEG_Session)

coords = [xyz
          for session in sessions
          for electrode in session
          for xyz in electrode.location
          if electrode.matches(atlas)]
color = lambda xyz:"red" if atlas.coordinate_selected(siibra.spaces['mni152'],xyz) else "gray"
from nilearn import plotting
plotting.view_markers(coords,list(map(color,coords)),marker_size=3)

### Other regional datasets

In [ ]:
features = atlas.get_features(siibra.modalities.EbrainsRegionalDataset)
for f in features:
    print(f"{f.name}\n{f.url}\n")